## Step 03 Setup Snowflake

During this step we will create our demo environment. Update the SQL variables below with your GitHub username and Personal Access Token (PAT) as well as with your forked GitHub repository information.

In [ ]:
SET MY_USER = CURRENT_USER();

SET GITHUB_SECRET_USERNAME = 'ketki-mude';
SET GITHUB_SECRET_PASSWORD = 'xxx';
SET GITHUB_URL_PREFIX = 'https://github.com/ketki-mude';
SET GITHUB_REPO_ORIGIN = 'https://github.com/ketki-mude/NOAA_Weather_Data.git';

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the account level objects (ACCOUNTADMIN part)
-- ----------------------------------------------------------------------------

USE ROLE ACCOUNTADMIN;

-- Roles
CREATE OR REPLACE ROLE NOAA_ROLE;
GRANT ROLE NOAA_ROLE TO ROLE SYSADMIN;
GRANT ROLE NOAA_ROLE TO USER IDENTIFIER($MY_USER);

GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE NOAA_ROLE;
GRANT EXECUTE TASK ON ACCOUNT TO ROLE NOAA_ROLE;
GRANT EXECUTE MANAGED TASK ON ACCOUNT TO ROLE NOAA_ROLE;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE NOAA_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE TO ROLE NOAA_ROLE;

-- Databases
CREATE OR REPLACE DATABASE NOAA_DB;
GRANT OWNERSHIP ON DATABASE NOAA_DB TO ROLE NOAA_ROLE;

-- Warehouses
CREATE OR REPLACE WAREHOUSE NOAA_WH WAREHOUSE_SIZE = XSMALL, AUTO_SUSPEND = 300, AUTO_RESUME= TRUE;
GRANT OWNERSHIP ON WAREHOUSE NOAA_WH TO ROLE NOAA_ROLE;

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the database level objects
-- ----------------------------------------------------------------------------
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE NOAA_WH;
USE DATABASE NOAA_DB;

-- Schemas
CREATE OR REPLACE SCHEMA INTEGRATIONS;
CREATE OR REPLACE SCHEMA DEV_SCHEMA;
CREATE OR REPLACE SCHEMA PROD_SCHEMA;
CREATE OR REPLACE SCHEMA DEV_HARMONIZED_SCHEMA;
CREATE OR REPLACE SCHEMA DEV_ANALYTICS_SCHEMA;
CREATE OR REPLACE SCHEMA DEV_HARMONIZED_SCHEMA;
CREATE OR REPLACE SCHEMA DEV_ANALYTICS_SCHEMA;

USE SCHEMA INTEGRATIONS;

-- Secrets (schema level)
CREATE OR REPLACE SECRET DEMO_GITHUB_SECRET
  TYPE = password
  USERNAME = $GITHUB_SECRET_USERNAME
  PASSWORD = $GITHUB_SECRET_PASSWORD;

-- API Integration (account level)
-- This depends on the schema level secret!
CREATE OR REPLACE API INTEGRATION DEMO_GITHUB_API_INTEGRATION
  API_PROVIDER = GIT_HTTPS_API
  API_ALLOWED_PREFIXES = ($GITHUB_URL_PREFIX)
  ALLOWED_AUTHENTICATION_SECRETS = (DEMO_GITHUB_SECRET)
  ENABLED = TRUE;

-- Git Repository
CREATE OR REPLACE GIT REPOSITORY DEMO_GIT_REPO
  API_INTEGRATION = DEMO_GITHUB_API_INTEGRATION
  GIT_CREDENTIALS = DEMO_GITHUB_SECRET
  ORIGIN = $GITHUB_REPO_ORIGIN;

In [ ]:
-- Step 1: Use the appropriate warehouse, database, and schema
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE NOAA_WH;
USE DATABASE NOAA_DB;

-- Step 2: Create the EXTERNAL schema
CREATE OR REPLACE SCHEMA EXTERNAL;

-- Step 3: Create an external stage for the NOAA data in S3
CREATE OR REPLACE STORAGE INTEGRATION noaa_s3_integration
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::908027397718:role/snowflake_role'
  STORAGE_ALLOWED_LOCATIONS = ('s3://noaaclimatedata/weatherData/');

-- Step 4: Grant access to the stage and schema
GRANT USAGE ON INTEGRATION noaa_s3_integration TO ROLE NOAA_ROLE;
GRANT USAGE ON SCHEMA EXTERNAL TO ROLE NOAA_ROLE;
GRANT ALL PRIVILEGES ON SCHEMA NOAA_DB.EXTERNAL TO ROLE NOAA_ROLE;

USE ROLE NOAA_ROLE;
USE WAREHOUSE NOAA_WH;
USE DATABASE NOAA_DB;
USE SCHEMA EXTERNAL;

-- Step 5: Create the file format for CSV files in the external stage
CREATE OR REPLACE FILE FORMAT CSV_FORMAT 
  TYPE = 'CSV' 
  FIELD_OPTIONALLY_ENCLOSED_BY = '"' 
  PARSE_HEADER = TRUE;


## Step 04 Deploy to Dev

Finally we will use `EXECUTE IMMEDIATE FROM <file>` along with Jinja templating to deploy the Dev version of our Notebooks. We will directly execute the SQL script `scripts/deploy_notebooks.sql` from our Git repository which has the SQL commands to deploy a Notebook from a Git repo.

In [ ]:
USE ROLE NOAA_ROLE;
USE WAREHOUSE NOAA_WH;
USE SCHEMA NOAA_DB.INTEGRATIONS;

EXECUTE IMMEDIATE FROM @DEMO_GIT_REPO/branches/main/scripts/deploy_notebooks.sql
    USING (env => 'DEV', branch => 'dev');

## Step 05 Load Weather from S3 to Snowflake s3